In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [1]:
with open("../data/metadata.py") as fp:
    data = fp.read()
    METADATA = eval(data.replace("\n", ""))

In [3]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

In [4]:
def get_page_number(link):
    return int(link.split("/")[-2].split("-")[-1])

def get_page_link(num, link):
    return f"{link}page-{num}/"

def get_articles_div(page):
    return page.find_element(
        By.CLASS_NAME, "topictabpane").find_elements(By.CLASS_NAME, "clearfix")

def get_article_metadata(article, article_num):
    try:
        return (
            article.find_element(
                By.CLASS_NAME, "lazy-container").text, 
            article.find_element(
                By.CLASS_NAME, "lazy-container").find_element(By.TAG_NAME, "a").get_attribute("href")
            )
    except:
        
        return (
            article.find_element(
                By.TAG_NAME, "a").find_element(By.XPATH, f'//*[@id="newslist-{article_num}"]/span').text,
            article.find_element(
                By.TAG_NAME, "a").get_attribute("href")
        )

In [6]:
for data in METADATA:

    LINK = data["link"]
    TICKER = data["ticker"]
    print(LINK, TICKER)

    # Open LINK
    driver.get(LINK)

    last_page_button = driver.find_element(
    By.CLASS_NAME, "pagenation").find_elements(
        By.CLASS_NAME, "last"
    )[-1]

    total_pages = get_page_number(last_page_button.get_attribute("href"))

    data = []
    for page_num in range(1, total_pages+1):
        page_link = get_page_link(page_num, LINK)
        driver.get(page_link)
        articles = get_articles_div(driver)
        for i, article in enumerate(articles):
            data.append(get_article_metadata(article, i))
        
        break
    data = pd.DataFrame(data)
    data.columns = ["Date", "link"]
    data.to_csv(f"../data/checkpoints/checkpoint_{TICKER}.csv", index=False)


https://www.moneycontrol.com/news/tags/hdfc.html/news/ HDFC


In [7]:
driver.quit()